In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import TSNE
from ipywidgets import interactive, HBox, VBox

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from plotly.offline import init_notebook_mode, iplot
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
primeiroCiC = ['116301', '118001', '118010', '113034', '140481', '145971']
segundoCiC = ['116319', '115045', '118028', '118036', '113042']
terceiroCiC = ['118044', '118052', '113956', '113051', '117366']


primeiroLic1 =['116793', '117366', '145971', '147389', '194221']
segundoLic1 = ['113034', '116301', '125156']
terceiroLic1 = ['115045', '116319', '125172']

primeiroLic2 =['113492', '194221', '113450', '113476']
segundoLic2 = ['113034', '116319', '191027', '150649', '194280']
terceiroLic2 = ['115045', '113093', '117366', '117889']

primeiroEngComp =['113034', '113093', '113476', '117528', '118001', '118010']
segundoEngComp = ['118028', '118036', '113042', '115045', '116319']
terceiroEngComp =['113051', '117242', '170054', '201600', '206075']


primeiroMec = ['114626', '114634', '118010', '113476', '113034', '118001', '168891']
segundoMec = ['168891', '113093', '118028', '118036', '115045', '168874']
terceiroMec = ['113051', '118044', '118052', '168769', '116319']

primeiroRedes = ['113034', '113093', '118001', '118010', '167959']
segundoRedes = ['113042', '115045', '118028', '118036', '169676']
terceiroRedes = ['113051', '113301', '167983', '169749']


undropableCol = ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]#, 'Ingresso_unb'

In [5]:
df0 = pd.read_csv("aluno_quant_Materias_Media_cotistas_0.csv", index_col=0)
df1 = pd.read_csv("aluno_quant_Materias_Media_cotistas_1.csv", index_col=0)
df2 = pd.read_csv("aluno_quant_Materias_Media_nao_cotistas_2.csv", index_col=0)
df3 = pd.read_csv("aluno_quant_Materias_Media_cotistas_3.csv", index_col=0)
df4 = pd.read_csv("aluno_quant_Materias_Media_cotistas_4.csv", index_col=0)
df5 = pd.read_csv("aluno_quant_Materias_Media_cotistas_5.csv", index_col=0)

In [6]:
dfComp = df0[undropableCol+primeiroLic1+segundoLic1+terceiroLic1].fillna(-1)
dfRedes = df1[undropableCol+primeiroRedes+segundoRedes+terceiroRedes].fillna(-1)
dfCic = df2[undropableCol+primeiroCiC+segundoCiC+terceiroCiC].fillna(-1) 
dfMec = df3[undropableCol+primeiroMec+segundoMec+terceiroMec].fillna(-1) 
dfEng = df4[undropableCol+primeiroEngComp+segundoEngComp+terceiroEngComp].fillna(-1) 

In [7]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [23]:
def formatColor(color):
    return 'rgb('+str(int(color[0]*255))+','+str(int(color[1]*255))+','+str(int(color[2]*255))+')'


    
def tsne(df, currentLabel = 'saiu'):
    originalMap = df.copy()
    originalMap = originalMap.reset_index()
    df = df.reset_index().drop(['index'], axis=1)
    X = df.values
    X_embedded = TSNE(n_components=2).fit_transform(X)


    data = df
    Y_sklearn = X_embedded
    y = data[currentLabel].values
    numColors = len(data[currentLabel].unique())

    cmap = get_cmap(numColors+1)
    colorList = [cmap(i) for i in range(1, numColors+1)]


    traceArr = []
    mapPoints = []
    for lab, col in zip(data[currentLabel].unique(),
                        colorList):
        mapPoints.append(data[y==lab].index)

        trace1 = go.Scatter(
            x=Y_sklearn[y==lab, 0],
            y=Y_sklearn[y==lab, 1],
            mode='markers',
            name = str(lab),
            marker=dict(
                size=5,
                color=formatColor(col),
                opacity=0.8
            ),

        )
        traceArr.append(trace1)
    f = go.FigureWidget(data=traceArr, layout=go.Layout(
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0
            )
        ))

    t = go.FigureWidget([go.Table(
    header=dict(values=["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

    def getSelector(index):
        def selection_fn(trace, points, selector):
            if (points.trace_index == 0):
                t.data[0].cells.values = [df.loc[mapPoints[index][points.point_inds]][col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]]
            else:
                w = [df.loc[mapPoints[index][points.point_inds]][col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]]
                s = t.data[0].cells.values
                w = pd.DataFrame(w).T
                a = pd.DataFrame([pd.Series(x) for x in list(s)]).T
                a.columns = w.columns
                t.data[0].cells.values = a.append(w).T
#             print(originalMap.loc[mapPoints[index][points.point_inds]])
        return selection_fn

    for ind, el in enumerate(f.data):
        el.on_selection(getSelector(ind))

    display(VBox((f,t)))


In [24]:
tsne(dfCic.copy())

    'data': [{'marker': {'color': 'rgb(7,255,0)', 'opacity': 0.8, 'size': 5},
  …

      index  curso  saiu  Cotista  Raça  Sexo  Tipo de Escola  116301  118001  118010  113034  140481  145971  116319  115045  118028  118036  113042  118044  118052  113956  113051  117366
108  111878      2    -1        0     5     1               0     4.0     3.0     4.0     3.0     4.0     4.0     5.0     3.0     5.0     4.0     4.0     3.0     4.0    5.00     3.0     4.0
215  125750      2    -1        0     5     0              -1     5.0     3.0     4.0     5.0     4.0     4.0     4.0     5.0     5.0     5.0     5.0     4.0     4.0    4.00     5.0     4.0
239  128113      2    -1        0     5     0               0     5.0     3.0     5.0     5.0     4.0     4.0     5.0     3.0     4.0     4.0     4.0     4.0     4.0    5.00     3.0     5.0
246  131849      2    -1        0     5     0              -1     5.0     4.0     4.0     5.0     5.0     5.0     4.0     4.0     4.0     4.0     3.0     4.0     3.0    2.50     3.0     3.0
249  132064      2    -1        0     5     1     

In [10]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

In [22]:
df = dfCic.copy()
df = df.reset_index().drop(['index'], axis=1)
currentLabel = 'saiu'

a = df
X = a.values
X_embedded = TSNE(n_components=2).fit_transform(X)


data = a
Y_sklearn = X_embedded
y = data[currentLabel].values
numColors = len(data[currentLabel].unique())

cmap = get_cmap(numColors+1)
colorList = [cmap(i) for i in range(1, numColors+1)]


traceArr = []
mapPoints = []
for lab, col in zip(data[currentLabel].unique(),
                    colorList):
    mapPoints.append(data[y==lab].index)
    
    trace1 = go.Scatter(
        x=Y_sklearn[y==lab, 0],
        y=Y_sklearn[y==lab, 1],
        mode='markers',
        name = str(lab),
        marker=dict(
            size=5,
            color=formatColor(col),
            opacity=0.8
        ),

    )
    traceArr.append(trace1)
f = go.FigureWidget(data=traceArr, layout=go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    ))

t = go.FigureWidget([go.Table(
header=dict(values=["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"],
            fill = dict(color='#C2D4FF'),
            align = ['left'] * 5),
cells=dict(values=[df[col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]],
           fill = dict(color='#F5F8FF'),
           align = ['left'] * 5))])

definedOnce = 0
testTuple = []
testArray = None
def getSelector(index):
    def selection_fn(trace,points,selector):
        if (points.trace_index == 0):
            t.data[0].cells.values = [df.loc[mapPoints[index][points.point_inds]][col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]]
        else:
            w = [df.loc[mapPoints[index][points.point_inds]][col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]]
            s = t.data[0].cells.values
            w = pd.DataFrame(w).T
            a = pd.DataFrame([pd.Series(x) for x in list(s)]).T
            a.columns = w.columns
            t.data[0].cells.values = a.append(w).T
            
        print(df.loc[mapPoints[index][points.point_inds]])
    return selection_fn

for ind, el in enumerate(f.data):
    el.on_selection(getSelector(ind))
    
VBox((f,t))

    'data': [{'marker': {'color': 'rgb(7,255,0)', 'opacity': 0.8, 'size': 5},
  …

     curso  saiu  Cotista  Raça  Sexo  Tipo de Escola    116301    118001  118010  113034  140481  145971  116319    115045  118028  118036  113042    118044  118052  113956    113051  117366
42       2    -1        0     5     0               0  3.666667  3.000000     3.0     3.0    -1.0     3.0     5.0  2.666667     4.0     2.5     3.0  2.333333     3.0     4.0  2.333333     4.0
205      2    -1        0     5     0               0  4.000000  5.000000     4.0     4.0    -1.0    -1.0     5.0  3.000000     4.0     5.0     4.0  3.000000     4.0     4.0  3.000000     4.0
280      2    -1        0     5     0              -1  4.000000  3.000000     4.0     4.0     5.0     3.0     3.0  3.000000     3.0     3.0     1.0  3.000000     4.0     3.5  2.000000     2.0
285      2    -1        0     5     0               1  4.000000  3.000000     3.0     3.0    -1.0     5.0     3.5  3.000000     3.0     4.0     4.0  2.000000     3.0     3.5  2.500000     3.0
308      2    -1        0     5     0   

In [12]:
s = testTuple[0]
w = testTuple[1].copy()

IndexError: list index out of range

In [ ]:
w = pd.DataFrame(w).T
w

In [ ]:
a = pd.DataFrame([pd.Series(x) for x in list(s)]).T
a.columns = w.columns
a

In [ ]:
x = a.append(w)
tupple(x)